In [1]:
from PMA import PMA_1
from torch.utils.data import Dataset,DataLoader
from myDataset import MyDataset
import torch

model = PMA_1()

#加载模型，进行预测
model.load_state_dict(torch.load("/home/user/Programs/PMA_lihang/code_lihang/model/model_1.pth"))
model.eval()
#加载测试数据
path = "/home/user/Programs/PMA_lihang/data/data_v_type_test.csv"
dataset = MyDataset(path)
dataloader = DataLoader(dataset,batch_size=4,shuffle=True)
#预测，计算准确率
correct = 0
total = 0
print("start predict")
for i,data in enumerate(dataloader):
    inputs,labels = data
    labels = torch.squeeze(labels)
    outputs = model(inputs)
    predict = torch.max(outputs,1)[1]
    total += labels.size(0)
    correct += (predict == labels).sum()
print("correct:%d,total:%d,acc:%f"%(correct,total,correct/total))

加载word2vec模型
加载数据集
数据集加载完成
start predict
correct:4729,total:5672,acc:0.833745


In [3]:
from PMA import PMA
from torch.utils.data import Dataset,DataLoader
from myDataset import MyDataset
import torch

model = PMA()
model.load_state_dict(torch.load("/home/user/Programs/PMA_lihang/code_lihang/model/model.pth"))
model.eval()
#加载测试数据
path = "/home/user/Programs/PMA_lihang/data/data_att_type_eval.csv"
dataset = MyDataset(path)
dataloader = DataLoader(dataset,batch_size=4,shuffle=True)
#预测，计算准确率
correct = 0
total = 0
print("start predict")
for i,data in enumerate(dataloader):
    inputs,labels = data
    labels = torch.squeeze(labels)
    outputs = model(inputs)
    predict = torch.max(outputs,1)[1]
    total += labels.size(0)
    correct += (predict == labels).sum()
print("correct:%d,total:%d,acc:%f"%(correct,total,correct/total))

加载数据集
数据集加载完成
start predict
correct:5692,total:6719,acc:0.847150


In [6]:
import time
from myDataset import MyDataset
from torch.utils.data import DataLoader
from PMA import PMA_11kernel
import torch
import torch.nn as nn

import torch.optim as optim



##1和5 卷积核大小不一样

#加载数据
path = "/home/user/Programs/PMA_lihang/data/data_v_type_train.csv"
dataset = MyDataset(path)
dataloader = DataLoader(dataset,batch_size=64,shuffle=True)
#训练模型
model = PMA_11kernel()
#损失函数
criterion = nn.CrossEntropyLoss()
#优化器
optimizer = optim.Adam(model.parameters(),lr=0.00001)
#训练
print("start training")
loss_all = []
#初始化准确率
acc = 0
for epoch in range(20):
    start_time = time.time()
    loss_ = 0
    for i,data in enumerate(dataloader):
        inputs,labels = data
        #对labels进行one-hot编码，总共6类，现在的值是下标
        #labels = labels[0]
        # print("inshape",inputs.shape)
        #将labels减少一个维度
        labels = torch.squeeze(labels)
        optimizer.zero_grad()
        outputs = model(inputs)
        # print("outsahpe",outputs.shape)
        # print("labelshape",labels.shape)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        loss_ += loss.item()
    end_time = time.time()
    loss_all.append(loss_)
    print("epoch:%d,step:%d,loss:%f"%(epoch,i,loss_))
    print("time:%f"%(end_time-start_time))
    #验证模型，计算准确率
    model.eval()
    path = "/home/user/Programs/PMA_lihang/data/data_v_type_eval.csv"
    dataset = MyDataset(path)
    dataloader = DataLoader(dataset,batch_size=4,shuffle=True)
    correct = 0
    total = 0
    
    print("start eval")
    for i,data in enumerate(dataloader):
        inputs,labels = data
        labels = torch.squeeze(labels)
        outputs = model(inputs)
        predict = torch.max(outputs,1)[1]
        total += labels.size(0)
        correct += (predict == labels).sum()
    print("correct:%d,total:%d,acc:%f"%(correct,total,correct/total))
    #如果现在的准确率大于之前的准确率，保存模型
    if correct/total > acc:
        acc = correct/total
        torch.save(model.state_dict(),"/home/user/Programs/PMA_lihang/code_lihang/model/model_1_1k.pth")
        print("*************epoch:",epoch,"****save model***********")


# #将loss写入文件
# path= "/home/user/Programs/PMA_lihang/code_lihang/model/train_log_1.txt"
# f = open(path,'w')
# for i in range(len(loss_all)):
#     f.write(str(loss_all[i]))
#     f.write("\n")
    
# #保存模型
# torch.save(model.state_dict(),"/home/user/Programs/PMA_lihang/code_lihang/model/model_1.pth")



model = PMA_11kernel()
model.load_state_dict(torch.load("/home/user/Programs/PMA_lihang/code_lihang/model/model_1_1k.pth"))
model.eval()
#加载测试数据
path = "/home/user/Programs/PMA_lihang/data/data_v_type_test.csv"
dataset = MyDataset(path)
dataloader = DataLoader(dataset,batch_size=64,shuffle=True)
#预测，
correct = 0
total = 0
print("start predict")
resoults = []
#总共13类。计算每一类的精确率和召回率
#每一类的预测正确的个数
class_num = 11
corrects = [0 for i in range(class_num)]
#每一类的预测错误的个数
errors = [0 for i in range(class_num)]
#每一类的实际正确的个数
labels = [0 for i in range(class_num)]
for i,data in enumerate(dataloader):
    inputs,labels_ = data
    labels_ = torch.squeeze(labels_)
    outputs = model(inputs)
    predict = torch.max(outputs,1)[1]
    resoults.append(predict)
    total += labels_.size(0)
    correct += (predict == labels_).sum()
    #计算每一类的预测正确的个数
    for i in range(len(predict)):
        if predict[i] == labels_[i]:
            corrects[predict[i]] += 1
        else:
            errors[predict[i]] += 1
    #计算每一类的个数
    for i in range(len(labels_)):
        labels[labels_[i]] += 1
    # break
print("correct:%d,total:%d,acc:%f"%(correct,total,correct/total))
# print("resoults:",resoults)
print("corrects:",corrects)
print("errors:",errors)
print("labels:",labels)
#计算每一类的精确率和召回率
for i in range(len(corrects)):
    if corrects[i] == 0:
        precision = 0
    else:
        precision = corrects[i]/(corrects[i]+errors[i])
    if labels[i] == 0:
        recall = 0
    else:
        recall = corrects[i]/labels[i]
    print("第%d类的精确率为%f,召回率为%f"%(i,precision,recall))
#计算f1值，只计算精确率和召回率都不为0的类
f1 = 0
count = 0
for i in range(len(corrects)):
    if corrects[i] == 0:
        precision = 0
    else:
        count += 1
        precision = corrects[i]/(corrects[i]+errors[i])
    if labels[i] == 0:
        recall = 0
    else:
        recall = corrects[i]/labels[i]
    if precision == 0 or recall == 0:
        continue
    f1 += 2*precision*recall/(precision+recall)
f1 = f1/count
print("f1:",f1)



ImportError: cannot import name 'PMA_11kernel' from 'PMA' (/home/user/Programs/PMA_lihang/code_lihang/model/PMA.py)